In [1]:
import json

with open('documents-with-ids.json', 'rt') as f_in:
    documents = json.load(f_in)

In [2]:
from sentence_transformers import SentenceTransformer

/home/ahmedk/Courses/Bootcamps/llm-zoomcamp/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = 'multi-qa-MiniLM-L6-cos-v1'
model = SentenceTransformer(model_name)

In [4]:
v = model.encode('I just discovered the course. Can I still join?')

/home/ahmedk/Courses/Bootcamps/llm-zoomcamp/.venv/lib/python3.9/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 384,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [7]:
from tqdm.auto import tqdm

In [8]:
for doc in tqdm(documents):
    question = doc['question']
    text = doc['text']
    qt = question + ' ' + text

    doc['question_vector'] = model.encode(question)
    doc['text_vector'] = model.encode(text)
    doc['question_text_vector'] = model.encode(qt)

100%|██████████| 948/948 [01:35<00:00,  9.88it/s]


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:08<00:00, 109.55it/s]


In [10]:
query = 'I just discovered the course. Can I still join it?'

In [11]:
v_q = model.encode(query)

In [12]:
len(v_q)

384

In [13]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs


In [14]:
def question_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_vector', v_q, course)


In [15]:
import pandas as pd

In [16]:
df_ground_truth = pd.read_csv('ground-truth-data.csv')

In [17]:
ground_truth = df_ground_truth.to_dict(orient='records')

In [18]:
ground_truth[0]

{'question': 'When exactly does the course begin, including the date and time?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [19]:
def mrr(relevance_total):
    score = 0
    for line in relevance_total:
        if True in line:
            score += 1 / (relevance_total.index(line)+1)
    return score / len(relevance_total)

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)


In [21]:
def evaluate(ground_truth, search_function):
    relevance_total = []
    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)
    
    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total)
    }

In [23]:
evaluate(ground_truth, lambda q: question_vector_knn(q))

100%|██████████| 4615/4615 [01:57<00:00, 39.37it/s]


{'hit_rate': 0.7651137594799566, 'mrr': 0.6212443076784424}

In [24]:
def text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('text_vector', v_q, course)


In [25]:
evaluate(ground_truth, lambda q: text_vector_knn(q))

100%|██████████| 4615/4615 [01:53<00:00, 40.68it/s]


{'hit_rate': 0.8305525460455038, 'mrr': 0.6637778765419036}

In [26]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)


In [27]:
evaluate(ground_truth, lambda q: question_text_vector_knn(q))

100%|██████████| 4615/4615 [01:40<00:00, 45.69it/s]


{'hit_rate': 0.9120260021668473, 'mrr': 0.783978085913466}